In [20]:
import pandas as pd
import cv2
import dlib
from imutils import face_utils

In [7]:
df=pd.read_csv("../facial_expressions/data/legend.csv").drop("user.id",axis=1)

In [4]:
IMG_PATH="../facial_expressions/images/"

In [8]:
df

,image,emotion
0,facial-expressions_2868588k.jpg,anger
1,facial-expressions_2868585k.jpg,surprise
2,facial-expressions_2868584k.jpg,disgust
3,facial-expressions_2868582k.jpg,fear
4,Aaron_Eckhart_0001.jpg,neutral
...,...,...
14185,Tim_Pawlenty_0001.jpg,neutral
14186,Tim_Robbins_0001.jpg,neutral
14187,Tim_Robbins_0002.jpg,neutral
14188,Tim_Robbins_0003.jpg,neutral


In [25]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor(args["shape_predictor"])

NameError: name 'args' is not defined

In [21]:
def ret_keypoints(imname):
    image = cv2.imread(IMG_PATH+imname)
#     image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#     boxes = [(y,x+w,y+h,x) for (x,y,w,h) in faces]
    shape=face_utils.shape_to_np(predictor(gray,[0,0,350,350]))
    return shape.ravel()

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
emotions=["neutral", "anger", "disgust", "fear", "happy", "sadness"]

In [ ]:
model=Sequential()
model.add(Dense(256,activation='relu', input_shape=(68*2,)))